# Deploying a thai-embedder model (kornwtp/simcse-model-phayathaibert)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact your Cloud Pack for Data administrator and ask them for your account credentials

In [ ]:
!pip install sentence-transformers==3.0.0
!pip install -U ibm-watson-machine-learning

In [ ]:
import numpy as np
import requests
from ibm_watson_machine_learning import APIClient

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url` and `api_key`.

In [6]:
# For on premise

# wml_credentials = {
#     "username": username,
#     "apikey": api_key,
#     "url": url,
#     "instance_id": 'openshift',
#     "version": '4.6'
# }

# For Saas
url='https://us-south.ml.cloud.ibm.com'
api_key=''
wml_credentials = {
        "url": url,
        "apikey": api_key
    }

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.5/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [8]:
space_id = '81ef41ad-a285-4d94-a8db-a7863eee0872'

You can use `list` method to print all existing spaces.

In [9]:
client.spaces.list(limit=10)

------------------------------------  --------------------------------------  ------------------------
ID                                    NAME                                    CREATED
c0c2c707-16bf-4620-929c-b0b5dae3116c  hugging-face-level1                     2024-04-13T08:44:07.135Z
e01b6ae7-3154-42ba-9fd3-ea298462286c  deploy                                  2024-02-28T09:28:04.456Z
4952f856-65b4-4df7-8523-08a012674e68  dep                                     2024-02-09T08:23:16.229Z
5fb9392c-787b-43a3-91e3-49c3b0355379  Tuning Space                            2024-02-07T05:27:24.684Z
f45fc607-d0b7-47fe-bdd7-65933e349238  deploy_gov_test2                        2023-12-27T10:15:15.265Z
a241367f-54e4-4fc8-b60a-d36ccb9e6bb1  deploy_gov_test                         2023-12-27T10:14:50.217Z
1975323a-9edf-4466-a381-6c104e5522f5  deploy_sepi2                            2023-12-27T10:13:42.684Z
5905937b-648b-4595-bd0a-0e330f8b099d  watsonx-governance-production           2023-12-27T0

,ID,NAME,CREATED
0,c0c2c707-16bf-4620-929c-b0b5dae3116c,hugging-face-level1,2024-04-13T08:44:07.135Z
1,e01b6ae7-3154-42ba-9fd3-ea298462286c,deploy,2024-02-28T09:28:04.456Z
2,4952f856-65b4-4df7-8523-08a012674e68,dep,2024-02-09T08:23:16.229Z
3,5fb9392c-787b-43a3-91e3-49c3b0355379,Tuning Space,2024-02-07T05:27:24.684Z
4,f45fc607-d0b7-47fe-bdd7-65933e349238,deploy_gov_test2,2023-12-27T10:15:15.265Z
5,a241367f-54e4-4fc8-b60a-d36ccb9e6bb1,deploy_gov_test,2023-12-27T10:14:50.217Z
6,1975323a-9edf-4466-a381-6c104e5522f5,deploy_sepi2,2023-12-27T10:13:42.684Z
7,5905937b-648b-4595-bd0a-0e330f8b099d,watsonx-governance-production,2023-12-27T08:43:23.572Z
8,db643a93-e681-45de-b4d0-118f262e4bff,watsonx-governance-incubation-practice,2023-12-27T08:23:47.787Z
9,7716c0a1-d194-453d-b96e-07c332b26df7,watsonx-governance-bootcamp,2023-12-27T07:36:24.395Z


To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [10]:
client.set.default_space(space_id)

'SUCCESS'

## Model Prep

In [38]:
%%writefile environment.yml
channels:
  - empty
  - nodefaults
dependencies:
- pip:
  - sentence-transformers==3.0.0

Overwriting environment.yml


In [73]:
def my_embedding_function():
    from sentence_transformers import SentenceTransformer
    model_name = 'kornwtp/simcse-model-phayathaibert'  # Replace with the desired model name
    try:
        model = SentenceTransformer(model_name)
    except Exception as e:
        return {"error": str(e)}
    
    def score(payload):
        # we assume only one batch is sent
        sentences = payload['input_data'][0]['values'][0]
        try:
            embeddings = model.encode(sentences)
            return {
                'predictions': [
                    {
                        'fields': ['sentence', 'embedding'],
                        'values': [[sentence, embedding.tolist()] for sentence, embedding in zip(sentences, embeddings)]
                    }
                ]
            }
        except Exception as e:
            return {"error": str(e)}
    return score

# Example usage:
embedding_function = my_embedding_function()

# Example payload
payload = {
    'input_data': [
        {
            'values': [
                ["กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด", "กลุ่มเด็กชายกำลังเล่นฟุตบอลบนชายหาด"]
            ]
        }
    ]
}

result = embedding_function(payload)
print(result)

{'predictions': [{'fields': ['sentence', 'embedding'], 'values': [['กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด', [1.1157501935958862, -0.11118047684431076, -0.5575134754180908, 0.4732487201690674, 0.4931929111480713, -0.09102023392915726, 0.861914873123169, 1.1215388774871826, 1.0791155099868774, -0.08809658885002136, 0.6136459112167358, 0.9309087991714478, 0.372568815946579, -0.30432072281837463, -0.4859830439090729, 0.3027385175228119, 1.9552267789840698, -0.04576771333813667, -0.5975145697593689, 0.34542304277420044, -0.3229100704193115, 0.07778365164995193, 1.09093177318573, 1.8012211322784424, -1.43440842628479, 0.06704423576593399, 0.13818810880184174, -2.1496405601501465, -1.1925352811813354, -0.8418582081794739, -1.2444835901260376, 2.0788049697875977, 2.455841064453125, -1.7201402187347412, -1.0865296125411987, -0.06961306929588318, -0.43986082077026367, -0.12048091739416122, -0.3960813879966736, -0.5594748258590698, 1.6785088777542114, 0.01844458281993866, 0.5065414309501648, 0.7382798194

In [29]:
client.repository.list_functions()

----  ----  -------  ----  ----------  ----------------
GUID  NAME  CREATED  TYPE  SPEC_STATE  SPEC_REPLACEMENT
----  ----  -------  ----  ----------  ----------------


,GUID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT


In [22]:
 client.software_specifications.list()

----------------------------  ------------------------------------  ----  ------------  --------------------------
NAME                          ID                                    TYPE  STATE         REPLACEMENT
default_py3.6                 0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  retired       runtime-23.1-py3.10
autoai-ts_rt23.1-py3.10       01ce9391-1a79-5a33-94fb-2e134337f314  base  supported
kernel-spark3.2-scala2.12     020d69ce-7ac1-5e68-ac1a-31189867356a  base  retired
pytorch-onnx_1.3-py3.7-edt    069ea134-3346-5748-b513-49120e15d288  base  retired
tensorflow_rt23.1-py3.10      079a91e0-245f-5269-8926-3c20b28f37dc  base  supported
scikit-learn_0.20-py3.6       09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  retired       runtime-23.1-py3.10
spark-mllib_3.0-scala_2.12    09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  retired
pytorch-onnx_rt22.1-py3.9     0b848dd4-e681-5599-be41-b5f6fccc6471  base  retired       pytorch-onnx_rt23.1-py3.10
ai-function_0.1-py3.6         0cdb0f1e-5

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,retired,runtime-23.1-py3.10
1,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,supported,
2,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,retired,
3,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,retired,
4,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,supported,
5,scikit-learn_0.20-py3.6,09c5a1d0-9c1e-4473-a344-eb7b665ff687,base,retired,runtime-23.1-py3.10
6,spark-mllib_3.0-scala_2.12,09f4cff0-90a7-5899-b9ed-1ef348aebdee,base,retired,
7,pytorch-onnx_rt22.1-py3.9,0b848dd4-e681-5599-be41-b5f6fccc6471,base,retired,pytorch-onnx_rt23.1-py3.10
8,ai-function_0.1-py3.6,0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda,base,retired,runtime-23.1-py3.10
9,shiny-r3.6,0e6e79df-875e-4f24-8ae9-62dcc2148306,base,retired,


In [78]:
client.hardware_specifications.list() 

----------------------  ------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------
NAME                    ID                                    DESCRIPTION
ML                      5a1f0e64-e420-55ce-bd7a-f6d00bb942cf  A hardware specification providing 4 CPU cores and 32 GiB of memory.
V100x2                  a02f3ab5-6964-4f06-a870-c7cc69187895  2 V100 GPU
L                       a6c4923b-b8e4-444c-9f43-8a7ec3020110  A hardware specification providing 8 CPU cores and 32 GiB of memory.
Default Spark           ac59d20a-9c7c-4504-a853-788ef35969da  A hardware specification for Spark with 1 CPU and 4 GiB of memory for master and worker nodes, with 2 workers.
XXS                     b128f957-581d-46d0-95b6-8af5cd5be580  A hardware specification providing one CPU core and 2 GiB of memory.
M-Spark                 b2232f7a-bfad-4822-9bce-6ba1af49217a  A hardware specification for

,NAME,ID,DESCRIPTION
0,ML,5a1f0e64-e420-55ce-bd7a-f6d00bb942cf,A hardware specification providing 4 CPU cores...
1,V100x2,a02f3ab5-6964-4f06-a870-c7cc69187895,2 V100 GPU
2,L,a6c4923b-b8e4-444c-9f43-8a7ec3020110,A hardware specification providing 8 CPU cores...
3,Default Spark,ac59d20a-9c7c-4504-a853-788ef35969da,A hardware specification for Spark with 1 CPU ...
4,XXS,b128f957-581d-46d0-95b6-8af5cd5be580,A hardware specification providing one CPU cor...
5,M-Spark,b2232f7a-bfad-4822-9bce-6ba1af49217a,A hardware specification for Spark service wit...
6,M,c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24,A hardware specification providing 4 CPU cores...
7,XL-Spark,c1791762-1333-4dd3-b7bb-228ae287da31,A hardware specification for Spark service wit...
8,K80,cf70f086-916d-4684-91a7-264c49c6d425,A hardware specification providing 4 CPU cores...
9,XL,d0aa1ae8-a889-42e2-a099-041b604b9289,A hardware specification providing 16 CPU core...


In [57]:
meta_props = {
   client.package_extensions.ConfigurationMetaNames.NAME: "transformers",
   client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}
pkg_extn_details = client.package_extensions.store(meta_props, "./environment.yml")
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)


base_id = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
meta_props = {
   client.software_specifications.ConfigurationMetaNames.NAME: "default with sentence-transformer",
   client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS: [{'guid': pkg_extn_id}],
   client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {'guid': base_id}
}
sw_spec_details = client.software_specifications.store(meta_props)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)


# model_details = client.repository.store_model(
#     model=sample_saved_model_filename,
#     meta_props=metadata
# )

Creating package extensions
SUCCESS


In [74]:
function_props = {
    client.repository.FunctionMetaNames.NAME: 'simcse embedder-v2',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_id
}
function_details = client.repository.store_function(my_embedding_function, function_props)
function_id = client.repository.get_function_id(function_details)
print(function_id)

42e44646-1983-4ffd-8d77-49c400f75203


In [80]:
hardware_spec_id = client.hardware_specifications.get_id_by_name('M')


deployment_props = {
    client.deployments.ConfigurationMetaNames.NAME: 'simcse embedder deployment v3',
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id, 'num_nodes': 1}
}
deployment_details = client.deployments.create(function_id, deployment_props)
deployment_id = client.deployments.get_id(deployment_details)



#######################################################################################

Synchronous deployment creation for uid: '42e44646-1983-4ffd-8d77-49c400f75203' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
...............
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='cd1c4aa3-697f-4221-a05e-48a930d11dac'
------------------------------------------------------------------------------------------------




In [82]:
payload = {
    'input_data': [
        {
            'values': [
                ["กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด", "กลุ่มเด็กชายกำลังเล่นฟุตบอลบนชายหาด"]
            ]
        }
    ]
}

client.deployments.score(deployment_id, payload)

{'predictions': [{'fields': ['sentence', 'embedding'],
   'values': [['กลุ่มผู้ชายเล่นฟุตบอลบนชายหาด',
     [1.1157499551773071,
      -0.1111806258559227,
      -0.5575138330459595,
      0.4732488691806793,
      0.4931944012641907,
      -0.0910203829407692,
      0.861914873123169,
      1.1215389966964722,
      1.0791149139404297,
      -0.08809694647789001,
      0.613644540309906,
      0.9309089183807373,
      0.372568279504776,
      -0.3043205738067627,
      -0.48598262667655945,
      0.3027377128601074,
      1.9552265405654907,
      -0.04576759785413742,
      -0.5975145697593689,
      0.34542346000671387,
      -0.3229110836982727,
      0.07778410613536835,
      1.0909312963485718,
      1.8012220859527588,
      -1.4344077110290527,
      0.06704407185316086,
      0.13818763196468353,
      -2.1496400833129883,
      -1.1925362348556519,
      -0.8418589234352112,
      -1.2444837093353271,
      2.0788052082061768,
      2.4558396339416504,
      -1.720139861106